In [ ]:
%load_ext autoreload
%autoreload 2

# SMINA Self docking

In [ ]:
import os
from cbiprep.smina import RunSmina, CheckRMSD

In [ ]:
valid_dirs = []
for root, files, dirs in os.walk('work'):
    if os.path.exists(f'{root}/status'):
        status = open(f'{root}/status', 'rt').read()
    else:
        status = None
    if status in ['matrix']:
        valid_dirs.append(root)
valid_dirs

In [ ]:
count = 0
for d in valid_dirs:
    count += 1
    if os.path.exists(f'{d}/status'):
        status = open(f'{d}/status', 'rt').read()
    else:
        status = None
    if status == 'docked':
        continue

    for fname in os.listdir(d):
        if fname.endswith('_apo.pdb.gz'):
            pdb_code = fname[:4]
        if fname.endswith('.sdf') and len(fname) == 7:
            ligand_name = fname[:3]
    print(pdb_code, ligand_name)
    docked_sdf = RunSmina(pdb_code, ligand_name)
    rmsd_oname = f'work/{pdb_code}/rmsd'
    with open(rmsd_oname, 'wt') as out:
        for i, rmsd in enumerate(CheckRMSD(pdb_code, ligand_name, docked_sdf), start=1):
            print(i, rmsd)
            print(i, rmsd, file=out)
    open(f'{d}/status', 'wt').write('docked')